In [1]:
!pip install petals
!pip install datasets
!pip install transformers
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 101.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 KB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 115.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 84.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [2]:
from petals import DistributedBloomForCausalLM
from transformers import BloomTokenizerFast
from datasets import load_dataset
from datasets.combine import concatenate_datasets
import evaluate
from tqdm import tqdm

In [3]:
def generate_prompts(train_size, test_size):
    dataset = load_dataset("boolq")
    dataset = dataset.shuffle()
    
    true_dataset = dataset["train"].filter(lambda example: example["answer"] == True).select(range(int(train_size / 2)))
    false_dataset = dataset["train"].filter(lambda example: example["answer"] == False).select(range(train_size - int(train_size / 2)))
    
    train_dataset = concatenate_datasets([true_dataset, false_dataset])
    train_dataset = train_dataset.shuffle()
    
    test_dataset = dataset["validation"].select(range(test_size))
    
    prompt = [f'Question: {example["question"]}\nPassage: {example["passage"]}\nAnswer: {example["answer"]}\n' for example in train_dataset]
    
    prompt = "\n".join(prompt)
    
    test_prompts = [f'\nQuestion: {example["question"]}\nPassage: {example["passage"]}\nAnswer: ' for example in test_dataset]

    test_answers = [example["answer"] for example in test_dataset]
    
    return prompt, test_prompts, test_answers

In [4]:
model = DistributedBloomForCausalLM.from_pretrained("bigscience/bloom-petals")
tokenizer = BloomTokenizerFast.from_pretrained("bigscience/bloom-petals")

train_size = 50
test_size = 100
responses = []

train_prompt, test_prompts, answers = generate_prompts(train_size, test_size)
print(test_prompts)
for test_prompt in (pbar := tqdm(test_prompts)):
  prompt = train_prompt + "\n" + test_prompt
  inputs = tokenizer.encode(prompt, return_tensors="pt")
  outputs = model.generate(inputs, max_new_tokens=3)
  response = tokenizer.decode(outputs[0], skip_special_tokens=True).split("Answer: ")[-1].split("\n")[0]
  result = 1 if response == " True" else 0
  responses.append(result)


accuracy = evaluate.load("accuracy")
for response, answer in zip(responses, answers):
    accuracy.add(predictions=response, references=answer)
print(accuracy.compute())

Downloading:   0%|          | 0.00/641 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/7.19G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

...

Filter:   0%|          | 0/9427 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9427 [00:00<?, ? examples/s]

["\nQuestion: is delhi university and university of delhi same\nPassage: The University of Delhi, informally known as Delhi University (DU), is a collegiate public central university, located in New Delhi, India. It was founded in 1922 by an Act of the Central Legislative Assembly. As a collegiate university, its main functions are divided between the academic departments of the university and affiliated colleges. Consisting of three colleges, two faculties, and 750 students at its founding, the University of Delhi has since become India's largest institution of higher learning and among the largest in the world. The university currently consists of 16 faculties and 86 departments distributed across its North and South campuses. It has 77 affiliated colleges and 5 other institutes with an enrollment of over 132,000 regular students and 261,000 non-formal students. The Vice-President of India serves as the University's chancellor. Rajat Chaudhary has now become the President of the Univ



  0%|          | 0/15 [00:00<?, ?it/s]

  7%|▋         | 1/15 [01:23<19:25, 83.27s/it]

 13%|█▎        | 2/15 [02:44<17:49, 82.24s/it]

 20%|██        | 3/15 [04:01<15:54, 79.50s/it]

 27%|██▋       | 4/15 [05:15<14:14, 77.69s/it]

 33%|███▎      | 5/15 [06:38<13:14, 79.46s/it]

 40%|████      | 6/15 [07:59<11:58, 79.82s/it]

 47%|████▋     | 7/15 [09:38<11:28, 86.08s/it]

 53%|█████▎    | 8/15 [11:00<09:55, 85.01s/it]

 60%|██████    | 9/15 [12:16<08:13, 82.24s/it]

 67%|██████▋   | 10/15 [13:44<06:59, 83.88s/it]

 73%|███████▎  | 11/15 [15:20<05:50, 87.62s/it]

 80%|████████  | 12/15 [17:01<04:35, 91.68s/it]

 87%|████████▋ | 13/15 [18:32<03:02, 91.37s/it]

 93%|█████████▎| 14/15 [19:51<01:27, 87.71s/it]

100%|██████████| 15/15 [21:17<00:00, 85.18s/it]


{'accuracy': 0.4}
